In [1]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE


In [2]:
df=pd.read_csv("auto insurance.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

In [4]:
df.isnull().sum().sort_values(ascending=False)

_c39                           1000
age                               0
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage                   0
bodily_injuries                   0
witnesses                         0
police_report_available           0
total_claim_amount                0
injury_claim                      0
property_claim                    0
vehicle_claim                     0
auto_make                         0
auto_model                        0
auto_year                         0
fraud_reported                    0
authorities_contacted             0
months_as_customer                0
collision_type                    0
insured_zip                       0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable           

In [5]:
categorical_cols = df.select_dtypes(include=['object', 'category'])
categorical_cols.sample(5)

,policy_bind_date,policy_state,policy_csl,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,incident_date,incident_type,...,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,property_damage,police_report_available,auto_make,auto_model,fraud_reported
615,27-02-1994,IL,250/500,MALE,High School,tech-support,paintball,other-relative,27-02-2015,Multi-vehicle Collision,...,Total Loss,Other,WV,Springfield,2696 Cherokee Ridge,?,NO,Nissan,Pathfinder,N
932,25-12-1997,IL,250/500,MALE,JD,craft-repair,polo,other-relative,27-01-2015,Multi-vehicle Collision,...,Minor Damage,Other,NC,Springfield,9352 Washington Ave,?,YES,Honda,Civic,N
569,31-12-2012,IL,250/500,MALE,College,craft-repair,camping,own-child,24-02-2015,Single Vehicle Collision,...,Total Loss,Other,SC,Springfield,9007 Francis Hwy,NO,YES,BMW,3 Series,N
417,20-02-2009,OH,250/500,FEMALE,JD,priv-house-serv,exercise,unmarried,28-01-2015,Single Vehicle Collision,...,Total Loss,Ambulance,NY,Hillsdale,9910 Maple Ave,YES,YES,Jeep,Grand Cherokee,N
415,07-09-2006,OH,500/1000,FEMALE,High School,machine-op-inspct,basketball,unmarried,11-01-2015,Single Vehicle Collision,...,Total Loss,Other,WV,Northbrook,1929 Britain Drive,NO,NO,Suburu,Legacy,N


In [6]:
categorical_cols.columns

Index(['policy_bind_date', 'policy_state', 'policy_csl', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'incident_date', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_state', 'incident_city', 'incident_location',
       'property_damage', 'police_report_available', 'auto_make', 'auto_model',
       'fraud_reported'],
      dtype='object')

In [7]:
categorical_cols['property_damage'].unique()

array(['YES', '?', 'NO'], dtype=object)

In [8]:
categorical_cols['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])  # ensure it's in datetime format
categorical_cols['policy_year'] = categorical_cols['policy_bind_date'].dt.year

categorical_cols['incident_date'] = pd.to_datetime(df['incident_date'])  # ensure it's in datetime format
categorical_cols['incident_year'] = categorical_cols['incident_date'].dt.year

categorical_cols['year_diff'] = categorical_cols['incident_year']-categorical_cols['policy_year']



C:\Users\brago\AppData\Local\Temp\ipykernel_19544\895121403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_cols['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])  # ensure it's in datetime format
C:\Users\brago\AppData\Local\Temp\ipykernel_19544\895121403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_cols['policy_year'] = categorical_cols['policy_bind_date'].dt.year
C:\Users\brago\AppData\Local\Temp\ipykernel_19544\895121403.py:4: SettingWithCopyWarning: 
A valu

In [9]:
categorical_cols=categorical_cols.drop(["insured_hobbies","incident_date","policy_bind_date","incident_location"],axis=1)
categorical_cols.shape

(1000, 20)

In [10]:
categorical_cols["incident_severity"].value_counts()

Minor Damage      354
Total Loss        280
Major Damage      276
Trivial Damage     90
Name: incident_severity, dtype: int64

In [11]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,TargetEncoder

In [18]:
le=LabelEncoder()
ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
ord_en=OrdinalEncoder()
encoder=ce.CountEncoder(normalize=True)
te=TargetEncoder()

In [19]:
# Column groups
freq_columns = ["policy_state", "policy_csl", "incident_type", "incident_severity",
                "insured_relationship", "authorities_contacted", "incident_state", "incident_city"]

ohe_columns = ["insured_sex", "property_damage", "police_report_available", "incident_type", "collision_type"]

te_columns = ["auto_make", "auto_model"]

In [22]:
# Label encode target
categorical_cols["fraud_reported"] = le.fit_transform(categorical_cols["fraud_reported"])

# Frequency Encoding
for col in freq_columns:
    categorical_cols[col] = encoder.fit_transform(categorical_cols[col])


# Target Encoding
for col in te_columns:
    categorical_cols[col] = te.fit_transform(categorical_cols[[col]], categorical_cols["fraud_reported"])


In [23]:
# Custom encoding based on frequency
education_order = {
    'JD': 1,
    'High School': 2,
    'Associate': 3,
    'MD': 4,
    'Masters': 5,
    'PhD': 6,
    'College': 7  # Assuming it's the least frequent
}
severity_order = {
    'Trivial Damage': 1,
    'Minor Damage': 2,
    'Major Damage': 3,
    'Total Loss': 4
}

categorical_cols['incident_severity'] = categorical_cols['incident_severity'].map(severity_order)
categorical_cols['insured_education_level'] = categorical_cols['insured_education_level'].map(education_order)


In [ ]:
# from sklearn.preprocessing import OrdinalEncoder

# # Define the custom category orders
# education_categories = [['JD', 'High School', 'Associate', 'MD', 'Masters', 'PhD', 'College']]
# severity_categories = [['Trivial Damage', 'Minor Damage', 'Major Damage', 'Total Loss']]

# # Initialize encoders with the given category orders
# education_encoder = OrdinalEncoder(categories=education_categories)
# severity_encoder = OrdinalEncoder(categories=severity_categories)

# # Apply encoding (reshape to 2D because sklearn expects 2D input)
# categorical_cols['insured_education_level'] = education_encoder.fit_transform(categorical_cols[['insured_education_level']]).astype(int)

# categorical_cols['incident_severity'] = severity_encoder.fit_transform(categorical_cols[['incident_severity']]).astype(int)


In [24]:
categorical_cols

,policy_state,policy_csl,insured_sex,insured_education_level,insured_occupation,insured_relationship,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,property_damage,police_report_available,auto_make,auto_model,fraud_reported,policy_year,incident_year,year_diff
0,0.352,0.351,MALE,4,craft-repair,0.170,0.403,Side Collision,NaN,0.292,0.248,0.149,YES,YES,0.226130,0.278651,1,2014,2015,1
1,0.310,0.351,MALE,4,machine-op-inspct,0.177,0.094,?,NaN,0.292,0.110,0.134,?,?,0.290052,0.249820,1,2006,2015,9
2,0.352,0.349,FEMALE,6,sales,0.183,0.419,Rear Collision,NaN,0.292,0.262,0.149,NO,NO,0.220949,0.312148,0,2000,2015,15
3,0.338,0.351,FEMALE,6,armed-forces,0.141,0.403,Front Collision,NaN,0.292,0.023,0.152,?,NO,0.285037,0.249820,1,1990,2015,25
4,0.338,0.300,MALE,3,sales,0.141,0.094,?,NaN,0.091,0.262,0.152,NO,NO,0.189578,0.106799,0,2014,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.352,0.300,FEMALE,5,craft-repair,0.141,0.403,Front Collision,NaN,0.223,0.110,0.122,YES,?,0.244484,0.294537,0,1991,2015,24
996,0.338,0.349,FEMALE,6,prof-specialty,0.155,0.403,Rear Collision,NaN,0.223,0.248,0.145,YES,?,0.273955,0.294297,0,2014,2015,1
997,0.352,0.351,FEMALE,5,armed-forces,0.177,0.419,Side Collision,NaN,0.292,0.110,0.152,?,YES,0.238238,0.308031,0,2003,2015,12
998,0.338,0.300,MALE,3,handlers-cleaners,0.155,0.403,Rear Collision,NaN,0.198,0.262,0.152,?,YES,0.263804,0.289715,0,2011,2015,4


In [ ]:
numerical_cols = df.select_dtypes(include=['number'])
numerical_cols.sample(5)

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
352,65,26,360770,500,1285.03,3000000,457942,0,-41500,0,1,1,3,52200,5220,10440,36540,2011,NaN
466,126,34,628337,2000,1078.22,0,453274,54500,0,2,3,0,1,58410,10620,5310,42480,2007,NaN
672,61,25,757644,2000,988.06,0,460579,0,0,1,3,1,0,32320,4040,4040,24240,2000,NaN
522,220,42,726880,1000,1281.72,0,619811,33500,-49500,20,4,0,2,47740,4340,4340,39060,2005,NaN
969,398,55,844062,500,862.19,0,606858,69400,0,7,1,2,3,6600,600,1200,4800,2012,NaN


In [ ]:
numerical_cols=numerical_cols.drop(["policy_number","_c39"],axis=1)

In [ ]:
numerical_cols.isnull().sum().sort_values(ascending=False)

months_as_customer             0
number_of_vehicles_involved    0
vehicle_claim                  0
property_claim                 0
injury_claim                   0
total_claim_amount             0
witnesses                      0
bodily_injuries                0
incident_hour_of_the_day       0
age                            0
capital-loss                   0
capital-gains                  0
insured_zip                    0
umbrella_limit                 0
policy_annual_premium          0
policy_deductable              0
auto_year                      0
dtype: int64

In [ ]:
import seaborn as sns

In [ ]:
numerical_cols.sample(3)

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year
167,298,45,1000,1362.29,0,442598,0,-51100,18,3,0,0,64000,12800,6400,44800,1999
839,234,43,500,1257.04,0,448294,0,-48800,4,1,2,2,51600,10320,5160,36120,2011
727,39,22,500,1173.21,0,431202,0,0,14,1,0,1,86130,15660,7830,62640,2009


In [ ]:
sns.boxplot(data=df, x='age', y='fraud_')

AttributeError: 'Series' object has no attribute 'boxplot'